# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.iloc[:,4:]

Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    #remove punctuation characters and convert to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #stop words
    stop_words = stopwords.words("english")
    
    # tokenize text
    tokens = word_tokenize(text)
    
    #Reduce tokenz to their stems
    stemmed = [PorterStemmer().stem(w) for w in tokens]
    
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
   
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#Split train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

#train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)


#f1 score, precision and recall for each output category
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.63      0.46      0.53      1551
          1       0.84      0.91      0.88      4943
          2       0.25      0.37      0.30        35

avg / total       0.79      0.80      0.79      6529

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5426
          1       0.77      0.42      0.54      1103

avg / total       0.87      0.88      0.87      6529

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6503
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6529

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3821
          1       0.74      0.60      0.66      2708

avg / total       0.75      0.75      0.74      6529

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {'tfidf__use_idf':[True, False],
              'tfidf__norm': ('l1', 'l2', None),
              'vect__max_df': (0.5, 0.75, 1.0),
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
#train GridSearch
cv.fit(X_train, y_train)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5, score=0.23759191176470587, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5, score=0.24050245098039216, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.5, score=0.25137867647058826, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75, score=0.23759191176470587, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  5.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75, score=0.24050245098039216, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=0.75, score=0.25137867647058826, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.23759191176470587, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__norm=l1,

[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 402.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': [True, False], 'tfidf__norm': ('l1', 'l2', None), 'vect__max_df': (0.5, 0.75, 1.0), 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [10]:
# predict on test data
y_pred = cv.predict(X_test)

#f1 score, precision and recall for each output category
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.72      0.43      0.54      1551
          1       0.84      0.95      0.89      4943
          2       0.43      0.26      0.32        35

avg / total       0.81      0.82      0.80      6529

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5426
          1       0.82      0.47      0.60      1103

avg / total       0.89      0.89      0.88      6529

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6503
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6529

aid_related
             precision    recall  f1-score   support

          0       0.78      0.84      0.81      3821
          1       0.75      0.68      0.71      2708

avg / total       0.77      0.77      0.77      6529

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
from custom_transformer import StartingVerbExtractor
from sklearn.pipeline import FeatureUnion

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
#adding startverbExtra and changing to AdaBoostClassifier
pipeline2 = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])
            ),
            
            ('starting_verb', StartingVerbExtractor()
            )
        ])
        ),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)
                                     )
        )
    ])
    


In [18]:
#train Pipeline2
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))])

In [19]:
# predict on test data
y_pred = pipeline2.predict(X_test)

#f1 score, precision and recall for each output category
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.69      0.09      0.16      1551
          1       0.77      0.99      0.87      4943
          2       0.43      0.17      0.24        35

avg / total       0.75      0.77      0.70      6529

request
             precision    recall  f1-score   support

          0       0.91      0.96      0.93      5426
          1       0.74      0.50      0.60      1103

avg / total       0.88      0.89      0.88      6529

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6503
          1       0.00      0.00      0.00        26

avg / total       0.99      0.99      0.99      6529

aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3821
          1       0.76      0.61      0.68      2708

avg / total       0.76      0.76      0.75      6529

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [20]:
with open('model.pickle', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.